In [1]:
#installing necessary libraries
%pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.


In [3]:
#importing necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchmetrics
from torchmetrics.classification import Accuracy   
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss

In [4]:
#Loading the dataset
df= pd.read_csv(r'C:\Users\LENOVO\Downloads\IDS-2018_Intrusion.csv')
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773.0,202,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0.0,38,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527.0,385,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,02/03/2018 08:47:40,132,2,0,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141.0,517,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [5]:
df.shape

(1048575, 80)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  float64
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

In [7]:
#Checking for missing values
df.isnull().sum()

Dst Port         0
Protocol         0
Timestamp        0
Flow Duration    0
Tot Fwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max         0
Idle Min         0
Label            0
Length: 80, dtype: int64

In [8]:
#checking for duplicate values
df.duplicated().sum()

5459

In [9]:
# Remove whitespace from column names
df.columns = df.columns.str.strip()

In [10]:
df['Label'].value_counts()

Label
Benign    762384
Bot       286191
Name: count, dtype: int64

In [11]:
#dropping constant columns
const_cols = [c for c in df.columns if df[c].nunique(dropna=False) <= 1]
df = df.drop(columns=const_cols)

In [12]:
#Target mapping
df['Label'] = df['Label'].map({'Benign': 0, 'Bot':1})

In [13]:
#Handling missing/infinite values if any
df = df.replace([np.inf, -np.inf], np.nan)
numeric_cols = df.select_dtypes(include='number').columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

In [14]:
#Encoding categorical features
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
cat_cols = [c for c in cat_cols if c!='Label']
df = pd.get_dummies(df, columns=[c for c in cat_cols if df[c].nunique()<=20], drop_first=True)

In [15]:
#Feature Scaling
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [16]:
#Splitting the dataset into training and testing sets
X = df.drop(columns=['Label'])
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns count:", len(numeric_cols))
if 'Timestamp' in X_train.columns:
    print("Dropping 'Timestamp' column (non-numeric).")
    X_train = X_train.drop(columns=['Timestamp'], errors='ignore')
    X_test  = X_test.drop(columns=['Timestamp'],  errors='ignore')
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

Numeric columns count: 68
Dropping 'Timestamp' column (non-numeric).


In [35]:
device = 'cpu' if not torch.cuda.is_available() else 'cuda'
#Creating a Neural Network model
layer0= nn.Linear(X_train.shape[1], 512)
layer1= nn.Linear(512, 256)
layer2= nn.Linear(256, 128)
layer3= nn.Linear(128, 2)
#Weight initialization
nn.init.uniform_(layer0.weight)
nn.init.uniform_(layer1.weight)
nn.init.uniform_(layer2.weight)
nn.init.uniform_(layer3.weight)
model= nn.Sequential(
    layer0,
    nn.ReLU(),
    nn.Dropout(0.25),
    layer1,
    nn.ReLU(),
    nn.Dropout(0.25),
    layer2,
    nn.ReLU(),
    layer3
)
model = model.to(device)

In [36]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

model.apply(init_weights)

Sequential(
  (0): Linear(in_features=68, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.25, inplace=False)
  (3): Linear(in_features=512, out_features=256, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.25, inplace=False)
  (6): Linear(in_features=256, out_features=128, bias=True)
  (7): ReLU()
  (8): Linear(in_features=128, out_features=2, bias=True)
)

In [37]:
#Computing class weights to handle class imbalance
classes = np.unique(y_train)
cw = compute_class_weight('balanced', classes=classes, y=y_train)


In [38]:
#Choosing loss function
weight_tensor = torch.tensor(cw, dtype=torch.float, device=device)
criterion = nn.CrossEntropyLoss(weight=weight_tensor)

In [39]:
#Defining dataset
X_train_tensor = torch.tensor(X_train.select_dtypes(include=['float64']).values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.astype(float), dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset= TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=2048, shuffle=False)

In [40]:
X_batch, y_batch = next(iter(train_loader))
X_batch = X_batch.to(device)
with torch.no_grad():
    sample_out = model(X_batch)
print("Sample output shape:", sample_out.shape)

Sample output shape: torch.Size([1024, 2])


In [41]:
#Setting up optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=1e-5, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

In [42]:
#Running a training loop
max_epochs = 50
early_patience = 5
best_val_acc = -np.inf
patience_counter = 0
best_path = "best_model.pth"
metric = torchmetrics.Accuracy(task="multiclass", num_classes=2).to(device)
for epoch in range(1, max_epochs+1):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device).long().view(-1)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*X_batch.size(0)
    MTL= train_loss/len(train_loader.dataset)
    #Model validation
    model.eval()
    metric.reset()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device).long().view(-1)
            outputs = model(X_batch)
            preds = outputs.argmax(dim=1).cpu()
            metric.update(preds.cpu(), y_batch.cpu())
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()*X_batch.size(0)
    MVL= val_loss/len(val_loader.dataset)
    Accuracy= float(metric.compute())
    scheduler.step(Accuracy)
    if Accuracy > best_val_acc:
        best_val_acc = Accuracy
        torch.save(model.state_dict(), best_path)
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_patience:
            print(f"Early stopping at epoch {epoch}. Best val Acc: {best_val_acc:.4f}")
            break 
    print(f"Epoch {epoch:02d} | Train loss: {train_loss:.4f})) | Val loss: {val_loss:.4f} | Val Acc: {Accuracy:.4f}")
    print(f"Mean Training Loss: {MTL:.4f} | Mean Validation Loss: {MVL:.4f}")
model.load_state_dict(torch.load(best_path, map_location=device))

Epoch 01 | Train loss: 109536.3720)) | Val loss: 56116.5401 | Val Acc: 0.9578
Mean Training Loss: 0.1306 | Mean Validation Loss: 0.0669
Epoch 02 | Train loss: 53699.0389)) | Val loss: 33547.5164 | Val Acc: 0.9604
Mean Training Loss: 0.0640 | Mean Validation Loss: 0.0400
Epoch 03 | Train loss: 31820.7047)) | Val loss: 16083.4585 | Val Acc: 0.9960
Mean Training Loss: 0.0379 | Mean Validation Loss: 0.0192
Epoch 04 | Train loss: 20698.6535)) | Val loss: 10415.0357 | Val Acc: 0.9969
Mean Training Loss: 0.0247 | Mean Validation Loss: 0.0124
Epoch 05 | Train loss: 15265.9598)) | Val loss: 8566.7391 | Val Acc: 0.9971
Mean Training Loss: 0.0182 | Mean Validation Loss: 0.0102
Epoch 06 | Train loss: 12783.7031)) | Val loss: 7633.9724 | Val Acc: 0.9976
Mean Training Loss: 0.0152 | Mean Validation Loss: 0.0091
Epoch 07 | Train loss: 10933.1377)) | Val loss: 6752.4373 | Val Acc: 0.9978
Mean Training Loss: 0.0130 | Mean Validation Loss: 0.0080
Epoch 08 | Train loss: 9606.9272)) | Val loss: 6395.7592 

<All keys matched successfully>